# Training a Simple GAN with PyTorch
- GAN for Generating simple MNIST Images
- Good to have a GPU for faster training

In [1]:
!rm -r track
!mkdir track

In [2]:
# Torch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Torchvision Imports
from torchvision import datasets, transforms
from torchvision.utils import save_image

# Device configuration; Good to have a GPU for GAN Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

# Generator Network
- 4 Fully Connected Layers
- Output Dimension will be same as 28*28

In [11]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, g_output_dim)


    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), negative_slope=0.2)
        x = F.leaky_relu(self.fc2(x), negative_slope=0.2)
        x = F.leaky_relu(self.fc3(x), negative_slope=0.2)
        return torch.tanh(self.fc4(x))


# Discriminator Network
- 4 Fully Connected Layers
- Output is a Single Neuron, 1=Real Image, 0=Fake Image

In [12]:
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), negative_slope=0.2)
        x = F.leaky_relu(self.fc2(x), negative_slope=0.2)
        x = F.leaky_relu(self.fc3(x), negative_slope=0.2)
        return torch.sigmoid(self.fc4(x))


# Instantiating Models
- Using a random noise of 100 dimensions
- Using BCE Loss for Training
- Using Adam Optimizer for Training

In [13]:
# Models
z_dim = 100 # Dimension of the Input Noise to the Generator
mnist_dim = 28 * 28 # As we have 28x28 pixel images in MNIST
G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

# loss
criterion = nn.BCELoss()

# optimizer
lr = 0.0005
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

# Loading & Normalizing MNIST Dataset

In [14]:
# Batch Size for Training
batch_size = 100

# Transforming Images to Tensor and then Normalizing the values to be between 0 and 1, with mean 0.5 and std 0.5
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))])

# Loading the Training Dataset and applying Transformation
train_dataset = datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# Data Loader which will be used as input to the models.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Method for Training Discriminator

In [15]:
def train_discriminator(mnist_data_batch):
    #===================Discriminator-Trainer===================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = mnist_data_batch.view(-1, mnist_dim), torch.full((batch_size, 1), 0.90)
    x_real, y_real = x_real.to(device), y_real.to(device)

    # Loss for Real MNIST Data
    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)

    # Generate Data from Generator Network for Training
    z = torch.randn(batch_size, z_dim).to(device) # Random Noise as Input to G
    x_fake, y_fake = G(z), torch.full((batch_size, 1), 0.1).to(device)

    # Loss for Fake Data from Generator
    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)

    # Updating only D's weights, so training on D on total loss
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()

    return  D_loss.data.item()


# Method for Training Generator

In [16]:
def train_generator():
    #=======================Generator-Trainer=======================#
    G.zero_grad()

    # Generate Random Noise to use as Input to the Generator
    z = torch.randn(batch_size, z_dim).to(device)
    # The final label in this case in 1(True), i.e the Discriminator Model
    # Thinks these are real images therefore the losses for Generator
    # Network should update the weights of G in such a way that it produces more
    # Real looking images.
    y = torch.full((batch_size, 1), 0.90).to(device)

    # Generate Images
    G_output = G(z)
    # Get output from Discriminator for the Generated Images
    D_output = D(G_output)
    # Compute Loss
    G_loss = criterion(D_output, y)
    # G_loss = -D_output.mean()

    # Updating only G's weights
    G_loss.backward()
    G_optimizer.step()

    return G_loss.data.item()

# Main Training Loop

In [ ]:
# Total number of epochs for Training
n_epoch = 200

losses = []

for epoch in range(n_epoch):
    D_losses, G_losses = [], []
    # For Every Batch in the MNIST dataset
    # We train the Discriminator First, then the Generator
    for batch_idx, (mnist_input_data, _) in enumerate(train_loader):
        D_losses.append(train_discriminator(mnist_input_data))

        # Weight Clamping
        for p in D.parameters():
          p.data.clamp_(-0.01, 0.01)

        if batch_idx % 2 == 0:
          G_losses.append(train_generator())

    if epoch % 10 == 0:
        test_z = torch.randn(8, z_dim).to(device)
        generated = G(test_z)
        save_image(generated.view(generated.size(0), 1, 28, 28), f'./track/sample_images_{epoch}' + '.png')

    # Logging the Loss values from Discriminator and Generator
    loss_d = torch.mean(torch.FloatTensor(D_losses))
    loss_g = torch.mean(torch.FloatTensor(G_losses))
    losses.append((loss_d, loss_g))
    print(f'[{epoch}/{n_epoch}]: loss_d: {loss_d}, loss_g: {loss_g}')

[0/200]: loss_d: 0.8678494095802307, loss_g: 1.9354009628295898
[1/200]: loss_d: 0.6945022344589233, loss_g: 2.118706464767456
[2/200]: loss_d: 0.6817149519920349, loss_g: 2.1160173416137695
[3/200]: loss_d: 0.6955651044845581, loss_g: 2.1698546409606934
[4/200]: loss_d: 0.7377318143844604, loss_g: 2.09809947013855
[5/200]: loss_d: 0.7426372170448303, loss_g: 1.96819007396698
[6/200]: loss_d: 0.7336576581001282, loss_g: 1.99467933177948
[7/200]: loss_d: 0.7046687006950378, loss_g: 2.0825297832489014
[8/200]: loss_d: 0.7505753040313721, loss_g: 2.0209078788757324
[9/200]: loss_d: 0.7307223677635193, loss_g: 2.046144485473633
[10/200]: loss_d: 0.7518192529678345, loss_g: 1.891499400138855
[11/200]: loss_d: 0.7489187717437744, loss_g: 1.8791155815124512
[12/200]: loss_d: 0.7474170923233032, loss_g: 1.8137696981430054
[13/200]: loss_d: 0.7766727805137634, loss_g: 1.793921947479248
[14/200]: loss_d: 0.79848313331604, loss_g: 1.7094461917877197
[15/200]: loss_d: 0.8160367608070374, loss_g: 1

# Generating Images from our Trained Generator Network

In [ ]:
with torch.no_grad():
    test_z = torch.randn(8, z_dim).to(device)
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './track/sample_images_200' + '.png')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(generated.view(generated.size(0), 1, 28, 28).cpu()[2].permute(1,2,0))

# Plotting Losses from the Training

In [ ]:
plt.plot(range(1, len(losses)+1), [_[0] for _ in losses], label='discriminator_loss')
plt.plot(range(1, len(losses)+1), [_[1] for _ in losses], label='generator_loss')
plt.legend()
plt.show()